In [0]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('..'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [2]:
!nvidia-smi

Sun Feb  9 04:01:44 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.48.02    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
!pip install transformers

     |████████████████████████████████| 481kB 2.7MB/s 
     |████████████████████████████████| 3.1MB 6.7MB/s 
     |████████████████████████████████| 1.0MB 20.7MB/s 
     |████████████████████████████████| 870kB 29.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=85b1f1e3e5bb0ec8e54c9e6999bb54b8c3678e28d656aa69b25c628339799541
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import tensorflow_hub as hub
import tensorflow as tf
import random
import pickle
import tensorflow.keras.backend as K
import gc
import time
import re
import os
import torch
from scipy.stats import spearmanr
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import KFold,StratifiedKFold
from scipy.stats import spearmanr
from math import floor, ceil
from transformers import AdamW,BertForSequenceClassification


In [0]:
sample_submission = pd.read_csv("/content/drive/My Drive/google-quest-challenge/sample_submission.csv")
test = pd.read_csv("/content/drive/My Drive/google-quest-challenge/test.csv")
train = pd.read_csv("/content/drive/My Drive/google-quest-challenge/train.csv")

MAX_SEQUENCE_LENGTH = 512

In [7]:
print('train shape =', train.shape)
print('test shape =', test.shape)

output_categories = list(train.columns[11:])
input_categories = list(train.columns[[1,2,5]])
print('\noutput categories:\n\t', output_categories)
print('\ninput categories:\n\t', input_categories)

train shape = (6079, 41)
test shape = (476, 11)

output categories:
	 ['question_asker_intent_understanding', 'question_body_critical', 'question_conversational', 'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer', 'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice', 'question_type_compare', 'question_type_consequence', 'question_type_definition', 'question_type_entity', 'question_type_instructions', 'question_type_procedure', 'question_type_reason_explanation', 'question_type_spelling', 'question_well_written', 'answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance', 'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 'answer_type_reason_explanation', 'answer_well_written']

input categories:
	 ['question_title', 'question_body', 'answer']


In [8]:
len(output_categories)

30

In [9]:
output_categories[21:]

['answer_helpful',
 'answer_level_of_information',
 'answer_plausible',
 'answer_relevance',
 'answer_satisfaction',
 'answer_type_instructions',
 'answer_type_procedure',
 'answer_type_reason_explanation',
 'answer_well_written']

In [0]:
def _get_masks(tokens, max_seq_length):
    """Mask for padding"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    return [1]*len(tokens) + [0] * (max_seq_length - len(tokens))

def _get_segments(tokens, max_seq_length):
    """Segments: 0 for the first sequence, 1 for the second"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    segments = []
    first_sep = True
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == '<SEP>':
            if first_sep:
                first_sep = False 
            else:
                current_segment_id = 1
    return segments + [0] * (max_seq_length - len(tokens))

def _get_ids(tokens, tokenizer, max_seq_length):
    """Token ids from Tokenizer vocab"""
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_ids = token_ids + [1] * (max_seq_length-len(token_ids))
    return input_ids

def _trim_input(title, question, answer, max_sequence_length, 
                t_max_len=30, q_max_len=233, a_max_len=234):

    t = tokenizer.tokenize(title)
    q = tokenizer.tokenize(question)
    a = tokenizer.tokenize(answer)
    
    t_len = len(t)
    q_len = len(q)
    a_len = len(a)

    if (t_len+q_len+a_len+15) > max_sequence_length:
        
        if t_max_len > t_len:
            t_new_len = t_len
            a_max_len = a_max_len + floor((t_max_len - t_len)/2)
            q_max_len = q_max_len + ceil((t_max_len - t_len)/2)
        else:
            t_new_len = t_max_len
      
        if a_max_len > a_len:
            a_new_len = a_len 
            q_new_len = q_max_len + (a_max_len - a_len)
        elif q_max_len > q_len:
            a_new_len = a_max_len + (q_max_len - q_len)
            q_new_len = q_len
        else:
            a_new_len = a_max_len
            q_new_len = q_max_len
            
            
        if t_new_len+a_new_len+q_new_len+15 != max_sequence_length:
            raise ValueError("New sequence length should be %d, but is %d" 
                             % (max_sequence_length, (t_new_len+a_new_len+q_new_len+15)))
        
        if t_len > t_new_len:
            ind1 = floor(t_new_len/2)
            ind2 = ceil(t_new_len/2)
            t = t[:ind1]+t[-ind2:]
        else:
            t = t[:t_new_len]

        if q_len > q_new_len:
            ind1 = floor(q_new_len/2)
            ind2 = ceil(q_new_len/2)
            q = q[:ind1]+q[-ind2:]
        else:
            q = q[:q_new_len]

        if a_len > a_new_len:
            ind1 = floor(a_new_len/2)
            ind2 = ceil(a_new_len/2)
            a = a[:ind1]+a[-ind2:]
        else:
            a = a[:a_new_len]
    
    return t, q, a

def _convert_to_bert_inputs(title, question, answer, tokenizer, max_sequence_length):
    """Converts tokenized input to ids, masks and segments for BERT"""
    
    stoken = ['<s>','<a1>','<a2>','<a3>','<a4>','<a5>','<a6>','<a7>','<a8>','<a9>'] + title + ['</s>','</s>'] + question + ['</s>','</s>'] + answer + ['</s>']

    input_ids = _get_ids(stoken, tokenizer, max_sequence_length)
    input_masks = _get_masks(stoken, max_sequence_length)
    input_segments = _get_segments(stoken, max_sequence_length)

    return [input_ids, input_masks, input_segments]

def compute_input_arays(df, columns, tokenizer, max_sequence_length):
    input_ids, input_masks, input_segments = [], [], []
    for _, instance in tqdm(df[columns].iterrows()):
        t, q, a = instance.question_title, instance.question_body, instance.answer

        t, q, a = _trim_input(t, q, a, max_sequence_length)

        ids, masks, segments = _convert_to_bert_inputs(t, q, a, tokenizer, max_sequence_length)
        input_ids.append(ids)
        input_masks.append(masks)
        input_segments.append(segments)
        
    return [np.asarray(input_ids, dtype=np.int32), 
            np.asarray(input_masks, dtype=np.int32), 
            np.asarray(input_segments, dtype=np.int32)]


def compute_output_arrays(df, columns):
    return np.asarray(df[columns])

In [0]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [0]:
class TextDataset(torch.utils.data.TensorDataset):

    def __init__(self, x_train, idxs, targets=None):
        self.input_ids = x_train[0][idxs]
        self.input_masks = x_train[1][idxs]
        self.input_segments = x_train[2][idxs]
        self.targets = targets[idxs] if targets is not None else np.zeros((x_train[0].shape[0], 30))

    def __getitem__(self, idx):
#         x_train = self.x_train[idx]
        input_ids =  self.input_ids[idx]
        input_masks = self.input_masks[idx]
        input_segments = self.input_segments[idx]

        target = self.targets[idx]

        return input_ids, input_masks, input_segments, target

    def __len__(self):
        return len(self.input_ids)

In [0]:
from transformers import BertTokenizer,BertConfig,get_linear_schedule_with_warmup, get_cosine_with_hard_restarts_schedule_with_warmup

In [0]:
from transformers import RobertaConfig, RobertaForSequenceClassification, RobertaTokenizer, get_cosine_with_hard_restarts_schedule_with_warmup

In [15]:
pretrained_weights = 'roberta-base'
tokenizer = RobertaTokenizer.from_pretrained(pretrained_weights)

In [16]:
special_tokens =['<a'+str(i)+'>' for i in range(1,10)]
special_tokens_dict = {"additional_special_tokens":special_tokens}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
print('We have added', num_added_toks, 'tokens')

We have added 9 tokens


In [0]:
output_categories_a = output_categories[21:]

In [18]:
output_categories_a

['answer_helpful',
 'answer_level_of_information',
 'answer_plausible',
 'answer_relevance',
 'answer_satisfaction',
 'answer_type_instructions',
 'answer_type_procedure',
 'answer_type_reason_explanation',
 'answer_well_written']

In [19]:
x_train = compute_input_arays(train, input_categories, tokenizer, MAX_SEQUENCE_LENGTH)
y_train = compute_output_arrays(train, output_categories_a)
x_test = compute_input_arays(test, input_categories, tokenizer, MAX_SEQUENCE_LENGTH)

6079it [00:16, 377.13it/s]
476it [00:01, 377.10it/s]


In [0]:
import torch
import torch.nn.functional as F
from torch import nn
from torch.nn import CrossEntropyLoss, MSELoss
from transformers import RobertaConfig, RobertaModel, RobertaForSequenceClassification, BertPreTrainedModel


ROBERTA_PRETRAINED_MODEL_ARCHIVE_MAP = {
    "roberta-base": "https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-pytorch_model.bin",
    "roberta-large": "https://s3.amazonaws.com/models.huggingface.co/bert/roberta-large-pytorch_model.bin",
    "roberta-large-mnli": "https://s3.amazonaws.com/models.huggingface.co/bert/roberta-large-mnli-pytorch_model.bin",
    "distilroberta-base": "https://s3.amazonaws.com/models.huggingface.co/bert/distilroberta-base-pytorch_model.bin",
    "roberta-base-openai-detector": "https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-openai-detector-pytorch_model.bin",
    "roberta-large-openai-detector": "https://s3.amazonaws.com/models.huggingface.co/bert/roberta-large-openai-detector-pytorch_model.bin",
}

class BertHead(nn.Module):
    def __init__(self, dp, feat_size):
        super(BertHead, self).__init__()
        self.dropout = nn.Dropout(dp)
        self.linear = nn.Linear(feat_size, 1)
    def forward(self, x):
        x = self.dropout(x)
        return self.linear(x)


class CustomizedRoberta(BertPreTrainedModel):
    config_class = RobertaConfig
    pretrained_model_archive_map = ROBERTA_PRETRAINED_MODEL_ARCHIVE_MAP
    base_model_prefix = "roberta"

    def __init__(self, config):
        super().__init__(config)

        self.config = config
        self.config.output_hidden_states=True
        self.roberta = RobertaModel(self.config)
        self.tqa_heads = nn.ModuleList([BertHead(0.1, self.config.hidden_size) for _ in range(self.config.num_labels)])

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
    ):
        outputs = self.roberta(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
        )

        tq_outputs = []
        for ix in torch.arange(self.config.num_labels):
            # retreve ith special token to do the task!
            feat = outputs[2][-1][:, ix+1, :]
            out = self.tqa_heads[ix](feat)
            tq_outputs += [out.view(-1, 1)]
        tq_outputs = torch.cat(tq_outputs, 1)

        outputs = (tq_outputs,) + outputs[2:]
        if labels is not None:
            if self.num_labels == 1:
                #  We are doing regression
                loss_fct = MSELoss()
                loss = loss_fct(logits.view(-1), labels.view(-1))
            else:
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            outputs = (loss,) + outputs

        return outputs  # (loss), logits, (hidden_states), (attentions)


In [0]:
import numpy as np

class callback:
    def __init__(self):
        self.score = list()
        self.model = list()
        self.data = list()
    
    def put(self, model,data, score):
        self.score.append(score)
        self.model.append(model)
        self.data.append(data)

    def get_model(self):
        ind = np.argmin(self.score)
        return self.model[ind]
    def get_data(self):
        ind = np.argmin(self.score)
        return self.data[ind]


In [22]:
NFOLDS = 5
BATCH_SIZE = 4
EPOCHS = 4
SEED = 4845
num_warmup_steps = 100
lr = 3e-5


gradient_accumulation_steps = 1
seed_everything(SEED)

model_list = list()


y_oof = np.zeros((len(train), 9))
test_pred = np.zeros((len(test), 9))


y_oof = np.zeros((len(train), 9))
test_pred = np.zeros((len(test), 9))

kf = KFold(n_splits=NFOLDS, shuffle=True)

test_loader = torch.utils.data.DataLoader(TextDataset(x_test, test.index),batch_size=BATCH_SIZE, shuffle=False)


for i, (train_idx, valid_idx) in enumerate(kf.split(x_train[0])):
    
    
    print(f'fold {i+1}')
    gc.collect()
    
    ## loader
    train_loader = torch.utils.data.DataLoader(TextDataset(x_train, train_idx, y_train),batch_size=BATCH_SIZE, shuffle=True)
    val_loader = torch.utils.data.DataLoader(TextDataset(x_train, valid_idx, y_train),batch_size=BATCH_SIZE, shuffle=False)
    

    t_total = len(train_loader)//gradient_accumulation_steps*EPOCHS

    roberta_config = RobertaConfig.from_pretrained(pretrained_weights) 
    roberta_config.num_labels = 9
    net = CustomizedRoberta.from_pretrained(pretrained_weights, config=roberta_config)
    net.resize_token_embeddings(len(tokenizer))
    net.cuda()
    
    loss_fn = torch.nn.BCEWithLogitsLoss()
    optimizer = AdamW(net.parameters(), lr = lr)
    scheduler = get_cosine_with_hard_restarts_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=t_total)  # PyTorch scheduler

    cb = callback()


    for epoch in range(EPOCHS):  

        start_time = time.time()
        avg_loss = 0.0
        net.train()

        for step, data in enumerate(train_loader):

            # get the inputs
            input_ids, input_masks, input_segments, labels = data


            pred = net(input_ids = input_ids.long().cuda(),
                             labels = None,
                             attention_mask = input_masks.cuda()
                            )[0]
            
            
            loss = loss_fn(pred, labels.cuda())
        
            avg_loss += loss.item()
            loss = loss / gradient_accumulation_steps
            loss.backward()

            if (step + 1) % gradient_accumulation_steps == 0:

                # Calling the step function on an Optimizer makes an update to its parameters
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()
                
                
        avg_val_loss = 0.0

        valid_preds = np.zeros((len(valid_idx), 9))
        true_label = np.zeros((len(valid_idx), 9))
        

        net.eval()
        for j,data in enumerate(val_loader):

            # get the inputs
            input_ids, input_masks, input_segments, labels = data
            pred = net(input_ids = input_ids.long().cuda(),
                             labels = None,
                             attention_mask = input_masks.cuda()
                            )[0]

            loss_val = loss_fn(pred, labels.cuda())
            avg_val_loss += loss_val.item()


            valid_preds[j * BATCH_SIZE:(j+1) * BATCH_SIZE] = pred.cpu().detach().numpy()
            true_label[j * BATCH_SIZE:(j+1) * BATCH_SIZE]  = labels


        elapsed_time = time.time() - start_time 

        score = 0
        for i in range(9):
          s = np.nan_to_num(
                    spearmanr(true_label[:, i], valid_preds[:, i]).correlation / 9)
          score += s

        

        print('Epoch {}/{} \t loss={:.4f}\t val_loss={:.4f}\t spearmanr={:.4f}\t time={:.2f}s'.format(epoch+1, EPOCHS, avg_loss/len(train_loader),avg_val_loss/len(val_loader),score, elapsed_time))

        cb.put(net,valid_preds,avg_val_loss/len(val_loader))



    model_list.append(cb.get_model())
    y_oof[valid_idx] = cb.get_data()
        


fold 1



Epoch 1/4 	 loss=0.3900	 val_loss=0.3683	 spearmanr=0.2862	 time=359.70s
Epoch 2/4 	 loss=0.3637	 val_loss=0.3622	 spearmanr=0.3408	 time=359.59s
Epoch 3/4 	 loss=0.3469	 val_loss=0.3640	 spearmanr=0.3556	 time=359.64s
Epoch 4/4 	 loss=0.3338	 val_loss=0.3667	 spearmanr=0.3532	 time=359.42s
fold 2
Epoch 1/4 	 loss=0.3872	 val_loss=0.3773	 spearmanr=0.2954	 time=359.30s
Epoch 2/4 	 loss=0.3605	 val_loss=0.3702	 spearmanr=0.3206	 time=360.21s
Epoch 3/4 	 loss=0.3427	 val_loss=0.3696	 spearmanr=0.3476	 time=358.53s
Epoch 4/4 	 loss=0.3281	 val_loss=0.3731	 spearmanr=0.3469	 time=358.67s
fold 3
Epoch 1/4 	 loss=0.3903	 val_loss=0.3671	 spearmanr=0.2889	 time=357.07s
Epoch 2/4 	 loss=0.3613	 val_loss=0.3644	 spearmanr=0.3323	 time=359.15s
Epoch 3/4 	 loss=0.3435	 val_loss=0.3644	 spearmanr=0.3501	 time=358.04s
Epoch 4/4 	 loss=0.3302	 val_loss=0.3687	 spearmanr=0.3452	 time=358.01s
fold 4
Epoch 1/4 	 loss=0.3880	 val_loss=0.3677	 spearmanr=0.2994	 time=358.84s
Epoch 2/4 	 loss=0.3608	 val_

In [0]:
with open("/content/drive/My Drive/qa_model/model0209_roberta_special_token_a.pkl","wb") as f:
    pickle.dump(model_list,f)

In [0]:
oof_score = 0
for i in range(30):
    oof_score += np.nan_to_num(
            spearmanr(y_train[:, i], y_oof[:, i]).correlation / 30)

In [0]:
print("Out of folds score = {}",oof_score)

Out of folds score = {} 0.3103592024675693


In [0]:
sample_submission.loc[:, output_categories] = test_pred
sample_submission.to_csv('/content/drive/My Drive/qa_submission/submission0129.csv', index=False)